In [1]:
import json
import torch
import functorch
import numpy as np
import pandas as pd

In [2]:
torch.__version__

'2.0.0'

In [3]:
torch.manual_seed(1943)

# Load training data

In [4]:
# training_data is a dict
# key is the name of an instance for hypernetwork training
# param is the parameters of MF model
# input is a matrix of N x 3 [Strike Price, Time to Maturity, Call/Put (+1,-1)]
# output is the price estimated by MF model
training_data = json.load(open('training_data.json'))
ids = sorted(training_data.keys())
# dataset (in Redis)
theta = torch.from_numpy(np.vstack([training_data[i]['param'] for i in ids]).astype(np.float32))
x = torch.from_numpy(np.stack([training_data[i]['input'] for i in ids]).astype(np.float32))
y = torch.from_numpy(np.stack([training_data[i]['output'] for i in ids]).astype(np.float32))

# Training the HyperNetwork

In [5]:
class HyperNetwork(torch.nn.Module):
    def __init__(self, hyper: torch.nn.Module, base: torch.nn.Module):
        super().__init__()
        self._hyper = hyper
        params_lookup = [[None, None, 0, 0]]
        for name, param in base.named_parameters():
            s = params_lookup[-1][-1]
            params_lookup.append([name, param.shape, s, s+np.prod(param.shape)])
        self._params_lookup = params_lookup[1:]
        buffers = {}
        self._call = lambda params, data: torch.func.functional_call(base, (params, buffers), (data,))

    def forward(self, x: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        params = self._hyper(x)
        params_dict = {}
        for i in self._params_lookup:
            params_dict[i[0]] = params[:, i[2]:i[3]].reshape(-1, *i[1])
        return torch.vmap(self._call, in_dims=(0, 0))(params_dict, z)

In [6]:
pricing_network = torch.nn.Sequential(
    torch.nn.Linear(3, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 1),
)

In [7]:
# compute the total number of parameters in the pricing network
n_params = sum([p.numel() for p in pricing_network.parameters()])

In [8]:
hyper_network = torch.nn.Sequential(
    torch.nn.Linear(4, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, n_params),
)

In [9]:
optim = torch.optim.Adam(hyper_network.parameters(), lr=1e-3)

In [10]:
module = HyperNetwork(hyper_network, pricing_network)

In [11]:
for I in range(1000):
    optim.zero_grad()
    rnd_idx = np.random.choice(16,8,replace=False)
    theta_, x_, y_ = theta[rnd_idx], x[rnd_idx], y[rnd_idx] # batch
    loss = torch.abs(module(theta_, x_)-y_).mean() # MAE
    loss.backward()
    optim.step()
    if I % 100 == 0:
        print(loss.detach().cpu().numpy())

1.7513721
0.04722009
0.050412063
0.035448443
0.04768291
0.025554935
0.013108632
0.013061664
0.0125591215
0.011982819


# Calibrating MF models by HyperNetwork

In [12]:
option_data = pd.read_json('option_data.json')
discount = option_data['discount_rate'].values.astype(np.float32)
forward_price = option_data['forward_price'].values.astype(np.float32)
norm_strike = option_data['strike'].values.astype(np.float32) / forward_price
option_price = option_data['option_price'].values.astype(np.float32)
norm_price = option_price / option_data['forward_price'].values.astype(np.float32)
put_call = (option_data['put_call']=='call').values.astype(np.float32)*2-1.0
tau = option_data['time_to_maturity'].values.astype(np.float32)

In [13]:
mf_model_param_unconstrained = torch.randn(4, requires_grad=True)

In [14]:
optim = torch.optim.Adam([mf_model_param_unconstrained], lr=1e-3)

In [15]:
for I in range(1000):
    
    H = torch.sigmoid(mf_model_param_unconstrained[0])*0.5      # (0, 0.5)
    rho = torch.tanh(mf_model_param_unconstrained[1])           # (-1,1)
    eta = torch.sigmoid(mf_model_param_unconstrained[2])*5.0    # (0, 5]
    v0 = torch.sigmoid(mf_model_param_unconstrained[3])         # (0, 1]
    
    mf_model_param = torch.stack([H, rho, eta, v0]).view(1,-1)
    
    pricing_network_input = torch.from_numpy(np.vstack([norm_strike, tau, put_call])).T.unsqueeze(0)
    model_pred = module(mf_model_param, pricing_network_input).squeeze(0,2)
    model_pred_denorm = model_pred * torch.from_numpy(forward_price).type_as(model_pred) * torch.from_numpy(discount).type_as(model_pred)
    model_output = torch.from_numpy(option_price)
    loss = torch.abs(model_pred_denorm-model_output).mean()

    optim.zero_grad()
    loss.backward()
    optim.step()
    
    if I % 100 == 0:
        print(loss.detach().cpu().numpy())

147.76677
128.19937
114.644165
101.46736
89.870895
81.63368
75.66382
71.07116
67.35002
64.4537
